# Gradient Descent Implementation

We use the public dataset on university admissions: Dataset http://www.ats.ucla.edu/stat/data/binary.csv -> here grades.txt

This dataset has three input features: GRE score, GPA, and the rank of the undergraduate school. 
Institutions with rank 1 have the highest prestige, those with rank 4 have the lowest.

In [ ]:
import pandas as pd
import numpy as np
admissions = pd.read_csv('grades.txt')
admissions.head()

In [ ]:
# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)
data.head()

In [ ]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
data.head()

In [ ]:
# 10 % Test data
np.random.seed(123)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

In [ ]:
print(data)
print(test_data)

In [ ]:
# Features and Labels / Targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [ ]:
targets

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoidPrime(x):
    return sigmoid(x)*(1-sigmoid(x))

np.random.seed(123)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1/n_features**.5, size=n_features)

epochs = 1000
learnrate = 0.3

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        h=np.dot(x, weights)
        output = sigmoid(h)
        error = y - output
        error_term = error * sigmoidPrime(h)
        del_w += error_term * x

    weights += learnrate * del_w / n_records

    # Print error
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        e = np.mean((out - targets) ** 2)
        print("Error: %(e).6f" % {"e":(e)})


# Calculate accuracy
test_out = sigmoid(np.dot(features_test, weights))
predictions = test_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Accuracy: %(acc).2f" % {"acc":(accuracy)})
